## Ground Segment Simulation Results

This notebook contains advanced plotting utilities for visualizing the results of a Ground Segment simulation with a
`ContactScheduler` block (either pure simulation or cosim). Use the code cells below to see the generated contact
schedule and contact report for each target-antenna pair.

## Setup

This notebook requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the root directory of this repository (two levels above this notebook) with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been
compromised, you can revoke it in the user settings interface on the Sedaro website.

Install required packages by running this code cell:

In [ ]:
%pip install -r requirements.txt

## Download Simulation Results

Here we will also get some relevant branch information for plotting purposes.

In [10]:
from utils import sedaroLogin, schedule_table, target_analytics
import plotly.express as px


# Settings
# Set this to the scenario branch ID of the simulation you want to visualize. For example, you can leave as if you want
# to see the pure sim results of the demo, or you can change it to the cosimulated branch ID.
pure_sim_branch_id = 'PPHLMVQdgb4LyvNBmfgTW4'
# Set this to the branch ID of the ground segment template used in the simulation. You may not need to change this.
ground_segment_template = 'PPHLMTljknVcsY6pnhJ5Qf' 
gs_agent_name = 'Atlas (Enterprise)'
job = None # If None, the most recent results will be downloaded

# Login
client = sedaroLogin()

# Load the results of the pure simulation
scenario = client.scenario(pure_sim_branch_id)
template = client.agent_template(ground_segment_template)
pure_sim = scenario.simulation.results(job_id=job)
ground_segment_results = pure_sim.agent(gs_agent_name)

Downloading...
...download complete!


## Contact Schedule

By accessing the simulation results of the `ContactScheduler` block, we can see the schedule that was generated. Note
that some of the colors are duplicated, but you can hover/zoom in on the interactive plot to see more information about
each contact.

In [11]:
plot_data = schedule_table(template, ground_segment_results)

fig = px.timeline(plot_data, x_start='start', x_end='end', y='antenna', color='target', title='Up/Downlink Schedule')
fig.update_yaxes(categoryorder='category descending')
# Alphabetize legend
fig.update_legends(traceorder='normal')
fig.show()

Same plot as above but with the y-axis and legend swapped

In [12]:
fig = px.timeline(plot_data, x_start='start', x_end='end', color='antenna', y='target', title='Up/Downlink Schedule (Per Spacecraft)')
fig.update_yaxes(categoryorder='category descending')
fig.show()

## Plot target-antenna availability

Here we will plot the target-antenna analytics. There are three categories of availability.

Access - This occurs when a spacecraft within line-of-sight of a particular antenna (including obscuration by an
elevation mask). This is an input to the `ContactScheduler`.

InFov - This occurs when a spacecraft is in the field of view associated with a particular antenna. In the case of this
simulation, the FoV will slew if an antenna has a scheduled contact.

Connected - This occurs when data is flowing between a spacecraft and a particular antenna.

In addition to the time series, which we plot below, there are also summary statistics (mean, max, min, number, etc.)
available for each definition of contact and gaps between contacts. For example, a target has a state variable
`accessMetrics` that will contain (among other things) the mean gap duration between access to that target. It also has
a `perCommDeviceAccessMetrics`, which contains the same statistics per antenna.

In [13]:
availability_data = target_analytics(ground_segment_results)
# This loop will create a new figure for each target
for target_name, data in availability_data.items():
    fig = px.timeline(data, x_start='start', x_end='end', y='antenna', color='type')
    fig.update_yaxes(categoryorder='total ascending')
    # set title
    fig.update_layout(title_text=f'{target_name} Access, In Fov, and Connected')
    fig.show()